# AutoGen Multi-Agent Propaganda Technique Analysis System

This notebook uses the AutoGen framework to create specialized agents for each language-technique combination, analyzing and summarizing LLM prompts and keywords for each technique.

## System Architecture
- **Coordinator Agent**: Coordinates the entire analysis workflow
- **Language-Technique Agents**: A dedicated agent for each language-technique combination
- **Reviewer Agent**: Reviews and iteratively optimizes results

## Workflow
1. Load API configuration
2. Scan all languages and techniques
3. Create agents for each combination
4. Collect articles from all frequency levels (high, medium, low)
5. Agents analyze and generate prompts and keywords
6. Iterate 3 times to optimize results
7. Save analysis results

## 1. Install and Import Dependencies

In [2]:
import os
import json
import autogen
from pathlib import Path
from collections import defaultdict
from typing import Dict, List, Tuple
import pandas as pd
from tqdm.notebook import tqdm
import time
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")
print(f"AutoGen version: {autogen.__version__}")

✅ Libraries imported successfully!
AutoGen version: 0.2.32


## 2. Configure Paths and Parameters

In [3]:
# Configure paths
BASE_DATA_PATH = "your_data_directory"
API_CONFIG_PATH = "your_api_config_file"
OUTPUT_DIR = "your_output_directory"

# Analysis parameters
ITERATION_COUNT = 3  # Number of iterations
FREQUENCY_LEVELS = ['high', 'medium', 'low']  # Frequency levels
MAX_ARTICLES_PER_TECHNIQUE = 30  # Maximum number of articles to analyze per technique

# Create output directory
Path(OUTPUT_DIR).mkdir(exist_ok=True)

print(f"Data path: {BASE_DATA_PATH}")
print(f"API config: {API_CONFIG_PATH}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Iteration count: {ITERATION_COUNT}")
print("✅ Configuration complete!")

Data path: your_data_directory
API config: your_api_config_file
Output directory: your_output_directory
Iteration count: 3
✅ Configuration complete!


## 3. Load API Configuration

In [4]:
def load_api_config(config_path):
    """Load API configuration file"""
    with open(config_path, 'r', encoding='utf-8') as f:
        config = json.load(f)
    return config

def setup_llm_config(api_config):
    """Setup LLM configuration (correct parameters)"""
    api_keys = api_config['api_keys']
    llm_config_list = []
    
    # OpenAI configuration
    if api_keys.get('openai_api_key'):
        llm_config_list.append({
            "model": "gpt-4o-mini",
            "api_key": api_keys['openai_api_key'],
        })
    
    # Google Gemini configuration (recommended: faster and more stable)
    if api_keys.get('google_api_key'):
        llm_config_list.append({
            "model": "gemini-1.5-flash",
            "api_key": api_keys['google_api_key'],
            "api_type": "google"
        })
    
    # Anthropic configuration
    if api_keys.get('anthropic_api_key'):
        llm_config_list.append({
            "model": "claude-3-haiku-20240307",
            "api_key": api_keys['anthropic_api_key'],
            "api_type": "anthropic"
        })
    
    return {
        "timeout": 180,  # Only keep timeout parameter (3 minutes)
        "cache_seed": 42,
        "config_list": llm_config_list,
        "temperature": 0.7,
    }

# Load configuration
api_config = load_api_config(API_CONFIG_PATH)
llm_config = setup_llm_config(api_config)

print(f"✅ API configuration loaded!")
print(f"Available models: {len(llm_config['config_list'])}")
print(f"Timeout setting: {llm_config['timeout']} seconds")
print(f"Primary models: {[cfg['model'] for cfg in llm_config['config_list']]}")

# Load configuration
api_config = load_api_config(API_CONFIG_PATH)
llm_config = setup_llm_config(api_config)

# Select model
llm_config['config_list'] = [
    {
        "model": "gpt-4o-mini",
        "api_key": api_config['api_keys']['openai_api_key']
    }
]


print(f"✅ API configuration loaded!")
print(f"Available models: {len(llm_config['config_list'])}")
print(f"Primary models: {[cfg['model'] for cfg in llm_config['config_list'][:3]]}")

✅ API configuration loaded!
Available models: 3
Timeout setting: 180 seconds
Primary models: ['gpt-4o-mini', 'gemini-1.5-flash', 'claude-3-haiku-20240307']
✅ API configuration loaded!
Available models: 1
Primary models: ['gpt-4o-mini']


## 4. Scan Data Structure

In [5]:
def scan_data_structure(base_path):
    """
    Scan data directory structure
    Returns: {language: {technique: {frequency: [article_paths]}}}
    """
    data_structure = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
    base_path = Path(base_path)
    
    if not base_path.exists():
        print(f"⚠️ Warning: Data path does not exist: {base_path}")
        return data_structure
    
    # Iterate through language folders
    for lang_dir in base_path.iterdir():
        if not lang_dir.is_dir():
            continue
        
        language = lang_dir.name
        
        # Iterate through frequency folders
        for freq_dir in lang_dir.iterdir():
            if not freq_dir.is_dir() or freq_dir.name not in FREQUENCY_LEVELS:
                continue
            
            frequency = freq_dir.name
            
            # Iterate through technique folders
            for tech_dir in freq_dir.iterdir():
                if not tech_dir.is_dir():
                    continue
                
                technique = tech_dir.name
                
                # Collect article files
                article_files = list(tech_dir.glob('*.txt'))
                data_structure[language][technique][frequency].extend(article_files)
    
    return data_structure

print("Scanning data structure...")
data_structure = scan_data_structure(BASE_DATA_PATH)

# Calculate statistics
total_articles = sum(
    sum(len(articles) for freq_articles in tech_data.values() for articles in freq_articles.values())
    for lang_data in data_structure.values()
    for tech_data in lang_data.values()
)

total_combinations = sum(
    len(tech_data) 
    for lang_data in data_structure.values() 
    for tech_data in lang_data.values()
)

print(f"\n✅ Data scan complete!")
print(f"Number of languages: {len(data_structure)}")
print(f"Language-technique combinations: {total_combinations}")
print(f"Total articles: {total_articles}")
print(f"\nLanguage list: {', '.join(sorted(data_structure.keys()))}")

Scanning data structure...

✅ Data scan complete!
Number of languages: 3
Language-technique combinations: 65
Total articles: 1284

Language list: en, po, ru


## 5. Define Agent System Prompts

In [6]:
def get_coordinator_prompt():
    """Coordinator Agent system prompt"""
    return """You are the Coordinator Agent responsible for managing the propaganda technique analysis workflow.

Your responsibilities:
1. Receive article collections for specific language-technique combinations
2. Delegate analysis tasks to specialized Language-Technique Agents
3. Collect and consolidate analysis results
4. Send results to the Reviewer Agent for quality checking
5. Iterate based on reviewer feedback

For each task, ensure:
- Clear communication with specialized agents
- Complete information transfer
- Tracking iteration progress
- Final result quality assurance
"""

def get_language_technique_agent_prompt(language, technique):
    """Generate specialized agent prompt for language-technique combination"""
    return f"""You are a specialized propaganda technique analyst for {technique} in {language} language.

Your task:
Analyze the provided article collection and generate:

1. **LLM Classification Prompt**
   - Clear, actionable definition of {technique}
   - How to identify this technique in {language} text
   - Specific linguistic markers or patterns
   - Few-shot examples (2-3 positive and negative cases)

2. **Keywords and Patterns**
   - High-frequency keywords characteristic of {technique}
   - Language-specific phrases and expressions
   - Structural patterns (sentence structures, discourse markers)

3. **Language-Specific Insights**
   - How {technique} manifests uniquely in {language}
   - Cultural or linguistic factors affecting detection
   - Comparison with other languages (if applicable)

Format your analysis clearly with these sections.
Be specific, evidence-based, and actionable.
"""

def get_reviewer_prompt():
    """Reviewer Agent system prompt"""
    return """You are the Quality Reviewer Agent responsible for evaluating and improving propaganda technique analyses.

For each analysis, check:

1. **Completeness**
   - Are all required sections present?
   - Is the LLM prompt clear and actionable?
   - Are keywords comprehensive and relevant?

2. **Quality**
   - Are definitions precise and accurate?
   - Are examples representative and diverse?
   - Are insights specific to the language-technique combination?

3. **Actionability**
   - Can the LLM prompt be directly used for classification?
   - Are keywords useful for feature engineering?
   - Are recommendations practical?

Provide:
- Specific feedback on what to improve
- Suggestions for missing elements
- Rating of current version (1-10)
- Whether another iteration is needed

Be constructive, specific, and actionable in your feedback.
"""

print("✅ Agent prompts defined!")

✅ Agent prompts defined!


## 6. Article Loading Function

In [7]:
def load_articles_for_technique(language, technique, data_structure, max_articles=MAX_ARTICLES_PER_TECHNIQUE):
    """
    Load articles for specific language-technique combination
    
    Args:
        language: Language code (e.g., 'en', 'po', 'ru')
        technique: Technique name (e.g., 'Appeal_to_Authority')
        data_structure: Data structure from scan_data_structure()
        max_articles: Maximum number of articles to load
    
    Returns:
        List of article texts
    """
    if language not in data_structure or technique not in data_structure[language]:
        return []
    
    all_articles = []
    
    # Collect articles from all frequency levels
    for frequency in FREQUENCY_LEVELS:
        article_paths = data_structure[language][technique].get(frequency, [])
        
        for path in article_paths:
            try:
                with open(path, 'r', encoding='utf-8') as f:
                    content = f.read().strip()
                    if content:  # Skip empty files
                        all_articles.append({
                            'text': content,
                            'frequency': frequency,
                            'filename': path.name
                        })
            except Exception as e:
                print(f"⚠️ Error reading {path}: {e}")
                continue
    
    # Limit number of articles
    if len(all_articles) > max_articles:
        # Prioritize: high frequency first, then medium, then low
        priority_order = {'high': 0, 'medium': 1, 'low': 2}
        all_articles.sort(key=lambda x: priority_order.get(x['frequency'], 3))
        all_articles = all_articles[:max_articles]
    
    return all_articles

print("✅ Article loading function defined!")

✅ Article loading function defined!


## 7. Single Language-Technique Analysis Function

In [8]:
def analyze_language_technique(language, technique, articles, llm_config, iteration_count=ITERATION_COUNT):
    """
    Analyze specific language-technique combination using multi-agent system
    
    Args:
        language: Language code
        technique: Technique name
        articles: Article list
        llm_config: LLM configuration
        iteration_count: Number of improvement iterations
    
    Returns:
        dict: Analysis results
    """
    try:
        # Create specialized analyst agent
        analyst = autogen.AssistantAgent(
            name=f"{language}_{technique}_analyst",
            system_message=get_language_technique_agent_prompt(language, technique),
            llm_config=llm_config,
        )
        
        # Create reviewer agent
        reviewer = autogen.AssistantAgent(
            name="reviewer",
            system_message=get_reviewer_prompt(),
            llm_config=llm_config,
        )
        
        # Create user proxy for interaction
        user_proxy = autogen.UserProxyAgent(
            name="coordinator",
            human_input_mode="NEVER",
            max_consecutive_auto_reply=10,
            code_execution_config=False,
        )
        
        # Prepare article sample (select representative examples)
        sample_size = min(10, len(articles))  # Use up to 10 articles for initial analysis
        article_sample = "\n\n---\n\n".join([
            f"Article {i+1} (Frequency: {art['frequency']}):\n{art['text'][:1000]}..." 
            for i, art in enumerate(articles[:sample_size])
        ])
        
        # Initial analysis request
        initial_message = f"""Analyze these {len(articles)} articles for {technique} in {language}.

Here is a sample of {sample_size} articles:

{article_sample}

Generate:
1. LLM Classification Prompt
2. Keywords and Patterns
3. Language-Specific Insights
"""
        
        # Start conversation
        user_proxy.initiate_chat(
            analyst,
            message=initial_message,
            max_turns=2
        )
        
        # Get initial analysis
        initial_analysis = analyst.last_message()["content"]
        
        # Iterative improvement
        current_analysis = initial_analysis
        feedback_history = []
        
        for iteration in range(iteration_count):
            # Get reviewer feedback
            review_message = f"""Review this analysis for {technique} in {language}:

{current_analysis}

Iteration {iteration + 1}/{iteration_count}
Previous feedback: {feedback_history[-1] if feedback_history else 'None'}
"""
            
            user_proxy.initiate_chat(
                reviewer,
                message=review_message,
                max_turns=2
            )
            
            reviewer_feedback = reviewer.last_message()["content"]
            feedback_history.append(reviewer_feedback)
            
            # Refine based on feedback
            if iteration < iteration_count - 1:  # Don't refine on last iteration
                refine_message = f"""Improve your analysis based on this feedback:

{reviewer_feedback}
"""
                user_proxy.initiate_chat(
                    analyst,
                    message=refine_message,
                    max_turns=2
                )
                current_analysis = analyst.last_message()["content"]
        
        return {
            'language': language,
            'technique': technique,
            'num_articles': len(articles),
            'initial_analysis': initial_analysis,
            'final_analysis': current_analysis,
            'reviewer_feedback': feedback_history,
            'status': 'success'
        }
        
    except Exception as e:
        return {
            'language': language,
            'technique': technique,
            'num_articles': len(articles),
            'error': str(e),
            'status': 'failed'
        }

print("✅ Analysis function defined!")

✅ Analysis function defined!


## 8. Run Complete Analysis (Main Loop)

In [ ]:
print("Starting multi-agent analysis...")
print("=" * 80)

analysis_results = []

# Iterate through all language-technique combinations
for language in tqdm(sorted(data_structure.keys()), desc="Languages"):
    lang_techniques = data_structure[language]
    
    # Create language-specific output directory
    lang_output_dir = Path(OUTPUT_DIR) / language
    lang_output_dir.mkdir(exist_ok=True)
    
    for technique in tqdm(sorted(lang_techniques.keys()), desc=f"{language} techniques", leave=False):
        print(f"\nProcessing: {language} - {technique}")
        
        # Load articles
        articles = load_articles_for_technique(language, technique, data_structure)
        
        if not articles:
            print(f"  ⚠️ No articles found, skipping...")
            continue
        
        print(f"  Loaded {len(articles)} articles")
        
        # Analyze
        result = analyze_language_technique(
            language=language,
            technique=technique,
            articles=articles,
            llm_config=llm_config,
            iteration_count=ITERATION_COUNT
        )
        
        analysis_results.append(result)
        
        # Save individual result
        if result['status'] == 'success':
            output_file = lang_output_dir / f"{technique}_analysis.md"
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(f"# {technique} in {language}\n\n")
                f.write(f"Number of articles analyzed: {result['num_articles']}\n\n")
                f.write("## Final Analysis\n\n")
                f.write(result['final_analysis'])
                f.write("\n\n## Reviewer Feedback History\n\n")
                for i, feedback in enumerate(result['reviewer_feedback'], 1):
                    f.write(f"### Iteration {i}\n\n")
                    f.write(feedback)
                    f.write("\n\n")
            print(f"  ✅ Saved: {output_file}")
        else:
            print(f"  ❌ Analysis failed: {result.get('error', 'Unknown error')}")
        
        # Brief pause to avoid API rate limits
        time.sleep(1)

print("\n" + "=" * 80)
print("Analysis complete!")
print("=" * 80)

## 9. Summary Statistics

In [ ]:
# Calculate statistics
success_count = sum(1 for r in analysis_results if r['status'] == 'success')
failed_count = sum(1 for r in analysis_results if r['status'] == 'failed')

print("\nAnalysis Summary")
print("=" * 80)
print(f"Total tasks: {len(analysis_results)}")
print(f"Successful: {success_count}")
print(f"Failed: {failed_count}")
print(f"Success rate: {success_count/len(analysis_results)*100:.1f}%")

# Statistics by language
lang_stats = defaultdict(lambda: {'success': 0, 'failed': 0})
for result in analysis_results:
    lang = result['language']
    status = result['status']
    lang_stats[lang][status] += 1

print("\nStatistics by language:")
for lang in sorted(lang_stats.keys()):
    stats = lang_stats[lang]
    total = stats['success'] + stats['failed']
    print(f"  {lang}: {stats['success']}/{total} successful")

# Create results DataFrame
results_df = pd.DataFrame([{
    'Language': r['language'],
    'Technique': r['technique'],
    'Articles': r.get('num_articles', 0),
    'Status': r['status'],
    'Error': r.get('error', '')
} for r in analysis_results])

print("\nResults preview:")
print("=" * 80)

# Method 1: Use print (most reliable)
print(results_df.head(10).to_string())

# Or

# Method 2: Use IPython display (if in Jupyter)
from IPython.display import display
display(results_df.head(10))

# Or

# Method 3: Direct DataFrame output (Jupyter auto-renders)
results_df.head(10)

print("=" * 80)

# Save summary results
results_df.to_csv(f"{OUTPUT_DIR}/analysis_summary.csv", index=False, encoding='utf-8-sig')
print(f"\n✅ Summary results saved to: {OUTPUT_DIR}/analysis_summary.csv")

# Additional: Display complete statistics
print("\nDetailed results:")
print("-" * 80)
for i, result in enumerate(analysis_results, 1):
    print(f"{i}. {result['language']} - {result['technique']}")
    print(f"   Status: {result['status']}")
    print(f"   Articles: {result.get('num_articles', 0)}")
    if result['status'] == 'success':
        print(f"   Analysis length: {len(result.get('final_analysis', ''))} characters")
    elif result['status'] == 'failed':
        print(f"   Error: {result.get('error', 'Unknown')}")
    print()

## 10. Generate Master Report

In [ ]:
def generate_master_report(results, output_path):
    """Generate master report"""
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write("# AutoGen Propaganda Technique Analysis - Master Report\n\n")
        f.write(f"Generated: {time.strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        # Overview
        f.write("## Analysis Overview\n\n")
        f.write(f"- Total tasks: {len(results)}\n")
        f.write(f"- Successful: {sum(1 for r in results if r['status'] == 'success')}\n")
        f.write(f"- Failed: {sum(1 for r in results if r['status'] == 'failed')}\n")
        f.write(f"- Iterations: {ITERATION_COUNT}\n")
        f.write(f"- Max articles per technique: {MAX_ARTICLES_PER_TECHNIQUE}\n\n")
        
        # Group by language
        f.write("## Results by Language\n\n")
        
        lang_results = defaultdict(list)
        for result in results:
            if result['status'] == 'success':
                lang_results[result['language']].append(result)
        
        for language in sorted(lang_results.keys()):
            f.write(f"### {language.upper()}\n\n")
            f.write(f"Techniques analyzed: {len(lang_results[language])}\n\n")
            
            for result in sorted(lang_results[language], key=lambda x: x['technique']):
                f.write(f"- **{result['technique']}** ({result['num_articles']} articles)\n")
                f.write(f"  - Detailed report: `{language}/{result['technique']}_analysis.md`\n")
            
            f.write("\n")
        
        # Failed tasks
        failed = [r for r in results if r['status'] == 'failed']
        if failed:
            f.write("## Failed Tasks\n\n")
            for result in failed:
                f.write(f"- {result['language']} - {result['technique']}: {result.get('error', 'Unknown error')}\n")
        
        f.write("\n---\n\n")
        f.write("## Usage Instructions\n\n")
        f.write("Detailed analyses for each language-technique combination are saved in corresponding language folders.\n\n")
        f.write("File format: `{language}/{technique}_analysis.md`\n\n")
        f.write("Each analysis file contains:\n")
        f.write("1. LLM classification prompts\n")
        f.write("2. Keywords and patterns\n")
        f.write("3. Language-specific insights\n")
        f.write("4. Reviewer feedback\n")

# Generate report
report_path = f"{OUTPUT_DIR}/MASTER_REPORT.md"
generate_master_report(analysis_results, report_path)

print(f"✅ Master report generated: {report_path}")

## 11. View Sample Result

In [ ]:
# Select a successful result to view
success_results = [r for r in analysis_results if r['status'] == 'success']

if success_results:
    sample = success_results[0]
    
    print("=" * 80)
    print(f"Sample analysis: {sample['language']} - {sample['technique']}")
    print("=" * 80)
    print(f"\nNumber of articles: {sample['num_articles']}")
    print(f"\nFinal analysis (first 1000 characters):\n")
    print(sample['final_analysis'][:1000])
    print("\n...")
else:
    print("⚠️ No successful analysis results available for display")

## 12. Export to JSON Format

In [ ]:
# Export all results as JSON
json_results = []
for result in analysis_results:
    if result['status'] == 'success':
        json_results.append({
            'language': result['language'],
            'technique': result['technique'],
            'num_articles': result['num_articles'],
            'analysis': result['final_analysis'],
            'reviewer_feedback': result['reviewer_feedback']
        })

json_path = f"{OUTPUT_DIR}/all_analyses.json"
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(json_results, f, ensure_ascii=False, indent=2)

print(f"✅ JSON format results saved: {json_path}")
print(f"Contains {len(json_results)} successful analyses")

## 13. Completion Summary

In [ ]:
print("\n" + "=" * 80)
print("🎉 AutoGen Multi-Agent Analysis Complete!")
print("=" * 80)

print(f"\n📁 Output directory: {OUTPUT_DIR}/")
print("\nGenerated files:")
print(f"  1. MASTER_REPORT.md - Master report")
print(f"  2. analysis_summary.csv - Results summary table")
print(f"  3. all_analyses.json - JSON format all analyses")
print(f"  4. [language]/[technique]_analysis.md - Detailed analysis for each technique")

print("\n📊 Statistics:")
print(f"  - Successful analyses: {success_count}")
print(f"  - Failed analyses: {failed_count}")
print(f"  - Total articles: {sum(r.get('num_articles', 0) for r in analysis_results)}")

print("\n💡 Next steps:")
print("  1. Review MASTER_REPORT.md for overall situation")
print("  2. Check detailed analyses in each language folder")
print("  3. Use generated LLM prompts to train classification models")
print("  4. Extract keywords for feature engineering")

print("\n" + "=" * 80)